In [1]:
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import pandas as pd

%matplotlib inline
import logging

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from cycler import cycler
from PIL import Image
from torchvision import datasets, transforms

import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

ModuleNotFoundError: No module named 'pytorch_metric_learning'

In [145]:
margin_param=0.2

In [146]:
# load x and y csv
x = pd.read_csv('x.csv')
# drop cod_modelo_color and des_filename
x = x.drop(columns=['cod_modelo_color', 'des_filename'])
y = pd.read_csv('y.csv')
x.head()

,des_age_Adult,des_agrup_color_eng_BLUE,des_agrup_color_eng_BROWN,des_agrup_color_eng_GREEN,des_agrup_color_eng_GREY,des_agrup_color_eng_ORANGE,des_agrup_color_eng_PINK,des_agrup_color_eng_PURPLE,des_agrup_color_eng_RED,des_agrup_color_eng_WHITE,...,des_product_type_Top,des_product_type_Totes bag,des_product_type_Trainers,des_product_type_Trenchcoat,des_product_type_Trousers,des_product_type_Turban,des_product_type_Umbrella,des_product_type_Vest,des_product_type_Wallet,des_sex_Female
0,True,False,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,False,True
1,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
2,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
3,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True


In [150]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class MyNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        return x

# Assuming you have a 159-dimensional input and want an output size of 20
input_size = 113
hidden_size1 = 64
hidden_size2 = 32
output_size = 20

# Create an instance of the model
model = MyNN(input_size, hidden_size1, hidden_size2, output_size)



In [151]:
class ItemDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x.values,dtype=torch.float32,requires_grad=True)
        self.y = torch.tensor(y.values).flatten()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = ItemDataset(x, y)

In [152]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.0001)
loss = losses.TripletMarginLoss(margin=margin_param)
miner= miners.TripletMarginMiner(margin=margin_param, type_of_triplets="hard")
sampler = samplers.MPerClassSampler(y, m=4, length_before_new_iter=len(train_dataset))

batch_size = 32
num_epochs = 10
dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
for i, (data,label) in enumerate(dataloader):
    optimizer.zero_grad()
    embeddings = model(data)
    hard_pairs = miner(embeddings, label)
    loss_value = loss(embeddings, label, hard_pairs)
    loss_value.backward()
    optimizer.step()

In [153]:
model.eval()

MyNN(
  (fc1): Linear(in_features=113, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=20, bias=True)
  (relu3): ReLU()
)

In [154]:
x_id = pd.read_csv('x.csv')
x_id["cod_modelo_color"]

0       53070992-01
1       51070902-02
2       53010605-09
3       53000578-01
4       53020724-14
           ...     
8736    57039043-96
8737    57039046-94
8738    57099074-02
8739    57039051-02
8740    57009044-99
Name: cod_modelo_color, Length: 8741, dtype: object

In [155]:
# calculate embeddings for all items
embeddings = model(torch.tensor(x.values,dtype=torch.float32,requires_grad=True))
# get cosine similarity between all items and item 0
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(embeddings, embeddings[0].unsqueeze(0))

In [156]:
# get argmax of cos_sim 
argmax = torch.argmax(cos_sim[1:])
argmax

cos_sim[argmax]

tensor(0.9926, grad_fn=<SelectBackward0>)

In [157]:
argmax.item()

65

In [ ]:
x_id.iloc[argmax.item()]

cod_modelo_color                                     57079082-TN
des_filename                datathon/images/2023_57079082_TN.jpg
des_age_Adult                                               True
des_age_Kids                                               False
des_agrup_color_eng_BLUE                                   False
                                            ...                 
des_product_type_Vest                                      False
des_product_type_Wallet                                    False
des_sex_Female                                              True
des_sex_Male                                               False
des_sex_Unisex                                             False
Name: 3484, Length: 124, dtype: object

In [ ]:
# dump model
torch.save(model.state_dict(), 'model.pt')

In [158]:
def get_embedding(model, x):
    return model(torch.tensor(x.values,dtype=torch.float32,requires_grad=True))

def get_recommendation(model, x, item_id,embeddings=None):
    if embeddings is None:  
        embeddings = get_embedding(model, x)
    
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    cos_sim = cos(embeddings, embeddings[item_id].unsqueeze(0))
    # return top 100
    top100 = torch.topk(cos_sim, 100)
    return top100

In [159]:


id = x_id.iloc[0]["cod_modelo_color"]
# get row_number of id
row_number = x_id[x_id["cod_modelo_color"] == id].index[0]

top100 = get_recommendation(model, x, row_number)

In [160]:
top100

torch.return_types.topk(
values=tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9990, 0.9990, 0.9990, 0.9990,
        0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990,
        0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990,
        0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990,
        0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990, 0.9990,
        0.9989, 0.9989, 0.9989, 0.9989, 0.9989, 0.9989, 0.9987, 0.9987, 0.9978,
        0.9978, 0.9977, 0.9977, 0.9977, 0.9977, 0.9977, 0.9977, 0.9977, 0.9977,
        0.9977], grad_fn=<TopkBackward0>),
indices=tensor([1014, 4276, 2738, 4581, 1245,

NameError: name 'df' is not defined